In [1]:
import numpy as np
import pandas as pd

In [2]:
#Import delle tabelle
loans_lenders = pd.read_csv(r'C://Users//ddoci//OneDrive//Documenti\\CS\\loans_lenders.csv')
loans = pd.read_csv(r'C://Users//ddoci//OneDrive//Documenti\\CS\\loans.csv')
lenders = pd.read_csv(r'C://Users//ddoci//OneDrive//Documenti\\CS\\lenders.csv')
country_stats = pd.read_csv(r'C://Users//ddoci//OneDrive//Documenti\\CS\\country_stats.csv')


## 1.Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [3]:
from itertools import chain
#imposto una funzione che restituisce una lista da una serie di stringhe, utilizzando come separatore la virgola
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))
# calcolo per ogni loan_id il numero di lenders che hanno contribuito
lens = loans_lenders['lenders'].str.split(',').map(len) 
#creo un nuovo dataframe avente nella colonna lenders un solo nome di lender per ogni 
#riga insieme al loan_id contenuto nell'omonima colonna
l_l = pd.DataFrame({'loan_id': np.repeat(loans_lenders['loan_id'], lens),
                    
                    'lenders': chainer(loans_lenders['lenders']),
                    })

#elimino i lenders che sono ripetuti più volte nel medesimo loan_id
loans_lenders_normalized=l_l.drop_duplicates(subset=['loan_id','lenders'])

loans_lenders_normalized

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499
...,...,...
1387430,1206425,trogdorfamily7622
1387430,1206425,danny6470
1387430,1206425,don6118
1387431,1206486,alan5175


## 2.For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.

In [4]:
pd.set_option('display.max_columns', None)

#mediante l'utilizzo di regex elimino dalle date i fusi orari, essendo essi superflui 
import re
loans['disburse_time'] = loans['disburse_time'].str.replace(r'\+0000', '')
loans['planned_expiration_time'] = loans['planned_expiration_time'].str.replace(r'\+0000', '')
#converto i formati delle date
loans['disburse'] = pd.to_datetime(loans['disburse_time'], format="%Y-%m-%d %H:%M:%S.%f")
loans['planned_time'] = pd.to_datetime(loans['planned_expiration_time'], format="%Y-%m-%d %H:%M:%S.%f")
#creazione di una nuova colonna contenente il numero di giorni tra il disburse time e il planned expiration time
loans['duration'] = loans['planned_time']-loans['disburse']
loans

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse,planned_time,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000,2013-12-22 08:00:00.000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,2013-12-22 08:00:00,2014-02-14 03:30:06,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000,2013-12-20 08:00:00.000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,2013-12-20 08:00:00,2014-03-26 22:25:07,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000,2014-01-09 08:00:00.000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09 08:00:00,2014-02-15 21:10:05,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000,2014-01-17 08:00:00.000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17 08:00:00,2014-02-21 03:10:02,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000,2013-12-17 08:00:00.000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17 08:00:00,2014-02-13 06:10:02,57 days 22:10:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419602,988180,NaN,NaN,NaN,NaN,400.0,400.0,funded,Tailoring,Services,NaN,KE,Kenya,NaN,shared,0.1,KES,138.0,2015-12-01 04:49:50.000 +0000,2016-01-02 01:00:03.000,2015-11-23 08:00:00.000,2015-12-28 15:44:18.000 +0000,14.0,16,4,2,"#Parent, #Repeat Borrower, #Woman Owned Biz",NaN,NaN,monthly,field_partner,2015-11-23 08:00:00,2016-01-02 01:00:03,39 days 17:00:03
1419603,988213,Perlita,English,"Perlita is 52 years old, married and has three...","Perlita is 52 years old, married and has three...",300.0,300.0,funded,Pigs,Agriculture,to buy feeds and other supplies to raise her pigs,PH,Philippines,"Numancia, Aklan",shared,0.1,PHP,145.0,2015-12-01 05:13:30.000 +0000,2016-01-02 16:40:07.000,2015-11-24 08:00:00.000,2015-12-22 10:37:06.000 +0000,14.0,12,1,1,"#Animals, #Elderly, #Repeat Borrower, #Woman O...",female,true,irregular,field_partner,2015-11-24 08:00:00,2016-01-02 16:40:07,39 days 08:40:07
1419604,989109,Okyeso Nyame Group,English,Okyeso Nyame group will begin its third cycle

## 3.Find the lenders that have funded at least twice.

In [5]:
#eseguo un merge per ottenere nello stesso dataframe il nome dei lenders e lo status dei prestiti 
loans_lenders_ = pd.merge(loans_lenders, loans[['loan_id', 'status']], on='loan_id', how='left')
#filtro il dataframe ottenendo solo i prestiti con lo status uguale a 'funded'
loans_funded = loans_lenders_[loans_lenders_['status'] == 'funded']
#conto quante volte il nome dello stesso lender compare e filtro tale risultato selezionando solo i lenders con almeno due 'funded'
lenders_funded =  loans_funded['lenders'].value_counts().reset_index(name='count').query('count >= 2')
lenders_funded.rename(columns={'index':'lenders'},inplace=True)
lenders_funded

,lenders,count
0,bob4739,1881
1,barbara5610,1749
2,blablablabla,1622
3,maryellen6459,1559
4,judith5696,862
...,...,...
5281,"howard4162, anders4748, hans8594, anders4748, ...",2
5282,peter90362979,2
5283,"alex39398685, alex39398685",2
5284,"judith5696, janandresen, dana9865",2


## 4.For each country, compute how many loans have involved that country as borrowers.

In [6]:
#filtro il dataframe eliminando eventuali date invalide
loans_not_null= loans[loans['planned_expiration_time'] > loans['disburse_time']]
#raggruppo per nazioni e conto il numero di volte dove tali nazioni compaiono come prestatori
country_borrowers= loans_not_null.groupby(['country_name']).count()['loan_id']
country_borrowers.reset_index().rename(columns={'loan_id':'count'})

,country_name,count
0,Afghanistan,1
1,Albania,3075
2,Armenia,12809
3,Azerbaijan,4026
4,Belize,98
...,...,...
79,Vanuatu,4
80,Vietnam,16203
81,Yemen,4184
82,Zambia,1038


## 5.For each country, compute the overall amount of money borrowed.

In [7]:
#raggruppo per nazioni e sommo per ognuna di esse il valore complessivo di soldi presi in prestito
money_borrowed= loans_not_null.groupby(['country_name']).sum()['loan_amount']
money_borrowed.reset_index().rename(columns= {'loan_amount':'money_borrowed'})

,country_name,money_borrowed
0,Afghanistan,6000.0
1,Albania,4307350.0
2,Armenia,20579275.0
3,Azerbaijan,6839400.0
4,Belize,65450.0
...,...,...
79,Vanuatu,9250.0
80,Vietnam,19907200.0
81,Yemen,3430100.0
82,Zambia,1658200.0


## 6.Like the previous point, but expressed as a percentage of the overall amount lent.

In [8]:
#eseguo un merge con i dataframe risultanti dai due punti precedenti
previous_points= pd.merge(country_borrowers,money_borrowed, on='country_name')
p_p= previous_points.reset_index().rename(columns={'loan_id':'count','loan_amount':'money_borrowed'})
#calcolo il totale dei soldi prestati
tot = p_p['money_borrowed'].sum()
#creo una nuova colonna, dove i valori presenti sono le percentuali dei soldi prestati ai singoli stati rispetto al totale dei soldi prestati
p_p['percentage'] = p_p['money_borrowed']/tot*100
p_p

,country_name,count,money_borrowed,percentage
0,Afghanistan,1,6000.0,0.000692
1,Albania,3075,4307350.0,0.496478
2,Armenia,12809,20579275.0,2.372031
3,Azerbaijan,4026,6839400.0,0.788330
4,Belize,98,65450.0,0.007544
...,...,...,...,...
79,Vanuatu,4,9250.0,0.001066
80,Vietnam,16203,19907200.0,2.294565
81,Yemen,4184,3430100.0,0.395364
82,Zambia,1038,1658200.0,0.191129


## 7.Like the three previous points, but split for each year (with respect to disburse time).

In [9]:
#recupero la soluzione del punto 4 e applico lo split per ogni anno rispetto al disburse time
point_4= loans_not_null.groupby([pd.Grouper(key='country_name'), pd.Grouper(key='disburse',freq='Y')]).count()['loan_id']
point_4.reset_index().rename(columns={'loan_id':'count'})


,country_name,disburse,count
0,Afghanistan,2015-12-31,1
1,Albania,2012-12-31,332
2,Albania,2013-12-31,507
3,Albania,2014-12-31,603
4,Albania,2015-12-31,638
...,...,...,...
472,Zimbabwe,2013-12-31,415
473,Zimbabwe,2014-12-31,2072
474,Zimbabwe,2015-12-31,600
475,Zimbabwe,2016-12-31,808


In [10]:
#recupero la soluzione del punto 5 e applico lo split per ogni anno rispetto al disburse time
point_5= loans_not_null.groupby([pd.Grouper(key='country_name'), pd.Grouper(key='disburse',freq='Y')]).sum()['loan_amount']
point_5.reset_index().rename(columns={'loan_amount':'money_borrowed'})


,country_name,disburse,money_borrowed
0,Afghanistan,2015-12-31,6000.0
1,Albania,2012-12-31,413000.0
2,Albania,2013-12-31,786950.0
3,Albania,2014-12-31,902550.0
4,Albania,2015-12-31,844375.0
...,...,...,...
472,Zimbabwe,2013-12-31,654025.0
473,Zimbabwe,2014-12-31,1302775.0
474,Zimbabwe,2015-12-31,723625.0
475,Zimbabwe,2016-12-31,788600.0


In [11]:
#aggiungo al df loans_not_null la colonna 'percentage', calcolando le percentuali dei soldi prestati ai singoli stati rispetto al totale dei soldi prestati 
loans_not_null['percentage'] = loans_not_null['loan_amount']/tot*100
#applico lo split relativo ad ogni anno rispetto al disburse time
point_6= loans_not_null.groupby([pd.Grouper(key='country_name'), pd.Grouper(key='disburse',freq='Y')]).sum()['percentage'] 
point_6.reset_index()

C:\Users\ddoci\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,country_name,disburse,percentage
0,Afghanistan,2015-12-31,0.000692
1,Albania,2012-12-31,0.047604
2,Albania,2013-12-31,0.090706
3,Albania,2014-12-31,0.104031
4,Albania,2015-12-31,0.097325
...,...,...,...
472,Zimbabwe,2013-12-31,0.075385
473,Zimbabwe,2014-12-31,0.150162
474,Zimbabwe,2015-12-31,0.083407
475,Zimbabwe,2016-12-31,0.090896


## 8.For each lender, compute the overall amount of money lent.

In [12]:
#creo un dataframe contenente il numero di lenders presenti per ogni loan_id 
lender = loans_lenders_normalized.groupby('loan_id').count().reset_index().rename(columns={'lenders' : 'count'})
#eseguo un merge per avere nel medesimo dataframe i valori creati dal df precedente, il 'loan_id' ed il valore dei prestiti
lender_merged = pd.merge(lender, loans_not_null, on='loan_id')[['loan_id', 'count', 'loan_amount']]
#calcolo per ogni lenders l'ammontare dei soldi prestati tenendo conto di un egual contributo dei lenders per il medesimo loan_id
lender_merged['money_lent'] = lender_merged['loan_amount'] / lender_merged['count']
lender_normalized = pd.merge(loans_lenders_normalized, lender_merged, on='loan_id', how='left')
amount_money_lent = lender_normalized.groupby('lenders')['money_lent'].sum().to_frame().reset_index()
amount_money_lent


,lenders,money_lent
0,000,1314.519396
1,00000,0.000000
2,0002,685.271014
3,00mike00,52.631579
4,0101craign0101,605.734354
...,...,...
1639021,zzanita,87.500000
1639022,zzcyna7269,55.357143
1639023,zzinnia,38.000000
1639024,zzmcfate,1323.375358


## 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known

In [13]:
#rimouvo i 'country code' nulli contenuti nel dataframe
country_not_null = lenders.loc[lenders['country_code'].notnull()].reset_index()
#isolo i valori che presentano 'country code' NaN in un dataframe apposito
country_null = lenders.loc[lenders['country_code'].isnull()].reset_index()
#calcolo il numero complessivo di 'country code' non nulli presenti nel dataframe
lenght_not_null = len(country_not_null.index)
#calcolo la distribuzione delle nazioni nel campione del dataframe dove il 'country code' è presente
sample = country_not_null[['index', 'country_code']].groupby('country_code').count().reset_index()
sample['percentage'] = sample['index']/lenght_not_null*100
#normalizzo le percentuali per scongiurare un noto problema della funzione np.random.choice, poichè senza normalizzare le percentuali dividendole per la somma delle probabilità non equivale ad 1
sample['percentage'] = sample['percentage']/ sample['percentage'].sum()
#sostituisco i 'country code' nulli con la distribuzione riscontrata nei valori aventi l'attributo 'country code' ed unisco i due dataframe 
country_null['country_code'] = np.random.choice(sample['country_code'], size=len(country_null.index), p = sample['percentage'])
country_lenders = pd.concat([country_not_null, country_null]).drop(columns='index')
#calcolo il valore dei soldi prestati per ogni nazione
country = country_lenders[['permanent_name','country_code']].rename(columns = {'permanent_name':'lenders'})
country_money_lent = pd.merge(country, amount_money_lent, on = 'lenders')
lent = country_money_lent.groupby('country_code').sum()['money_lent']
m_borrowed = money_borrowed.reset_index().rename(columns= {'loan_amount':'money_borrowed'})
#calcolo la differenza tra soldi prestati e soldi presi in prestito per ogni nazione
country = country_stats[['country_name','country_code']]
borrowed = pd.merge(m_borrowed, country, how = 'inner', on = 'country_name')
difference = pd.merge(lent, borrowed, on='country_code')
difference['Difference'] = difference['money_lent']-difference['money_borrowed']
difference[['country_name','country_code','money_lent','money_borrowed','Difference']]

,country_name,country_code,money_lent,money_borrowed,Difference
0,Afghanistan,AF,6444.631394,6000.0,4.446314e+02
1,Albania,AL,966.568647,4307350.0,-4.306383e+06
2,Armenia,AM,3022.172429,20579275.0,-2.057625e+07
3,Azerbaijan,AZ,2326.994961,6839400.0,-6.837073e+06
4,Burkina Faso,BF,169.735670,3989650.0,-3.989480e+06
...,...,...,...,...,...
70,Kosovo,XK,379.003215,3083025.0,-3.082646e+06
71,Yemen,YE,6979.648730,3430100.0,-3.423120e+06
72,South Africa,ZA,29445.461540,250250.0,-2.208045e+05
73,Zambia,ZM,684.002749,1658200.0,-1.657516e+06


## 10.Which country has the highest ratio between the difference computed at the previous point and the population?

In [14]:
#estraggo dal dataframe le colonne aventi come valori 'country code' e 'population' e le aggiungo al dataframe risultante dal punto precedente
pop_country = country_stats[['country_code','population']]
ratio_pop = pd.merge(difference, pop_country, how='inner', on = 'country_code')
#calcolo il ratio richiesto dal punto 10 e visualizzo la nazione con il ratio più eleveto
ratio_pop['ratio'] = ratio_pop['Difference'] / ratio_pop['population']
ratio_pop[ratio_pop['ratio']==ratio_pop['ratio'].max()]

,country_code,money_lent,country_name,money_borrowed,Difference,population,ratio
68,US,4.247183e+07,United States,28216050.0,1.425578e+07,324459463,0.043937


## 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [15]:
#calcolo gli abitanti non al di sotto della soglia di povertà
country_stats['population_above_poverty_line'] = (1- country_stats['population_below_poverty_line']/100) * country_stats['population']
#come per il punto precedente estraggo i valori di interesse e gli aggiungo al dataframe risultante del punto 9, calcolo il ratio richiestomi dal punto 11 e visualizzo la nazione con il ratio maggiore
poverty_line = country_stats[['country_code','population_above_poverty_line']]
ratio_poverty = pd.merge(difference, poverty_line, how='inner', on = 'country_code')
ratio_poverty['ratio'] = ratio_poverty['Difference'] / ratio_poverty['population_above_poverty_line']
ratio_poverty[ratio_poverty['ratio']==ratio_poverty['ratio'].max()]

,country_code,money_lent,country_name,money_borrowed,Difference,population_above_poverty_line,ratio
68,US,4.247183e+07,United States,28216050.0,1.425578e+07,2.754661e+08,0.051751


## 12.For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [17]:
# default='warn'
pd.options.mode.chained_assignment = None  
#partendo dal dataframe 'loans_not_null' isolo gli anni di inizio e fine dei prestiti e calcolo la differenza
df= loans_not_null
df['year_start'] = pd.DatetimeIndex(df['disburse']).year
df['year_end'] = pd.DatetimeIndex(df['planned_time']).year
df['years']= df['year_end'] - df['year_start']
tab= df[['loan_id','loan_amount','year_start','year_end','years','disburse','planned_time','duration']]
#creo due colonne aventi come valori l'ultimo giorno dell'anno di inizio prestito e il primo giorno dell'anno relativo all'ultimo anno di prestito
tab['year_start']=tab['year_start'].astype(int)
tab['year_end']=tab['year_end'].astype(int)
tab['month_s'] = 12
tab['day_s'] = 31
tab['first_year'] = pd.to_datetime((tab.year_start * 10000 + tab.month_s * 100 + 31).apply(str), format='%Y%m%d')
tab['month_e'] = 1
tab['day_e'] = 1
tab['last_year'] = pd.to_datetime((tab.year_end * 10000 + tab.month_e * 100 + 1).apply(str), format='%Y%m%d')
tab['disburse'] = pd.to_datetime(tab['disburse']).dt.date
tab['planned_time'] = pd.to_datetime(tab['planned_time']).dt.date
tab['delta_start'] = pd.to_datetime(tab['first_year']) - pd.to_datetime(tab['disburse'])
days_first_year=tab.delta_start.dt.days
tab['delta_end'] = pd.to_datetime(tab['planned_time']) - pd.to_datetime(tab['last_year'])
days_last_year=tab.delta_end.dt.days
pre_process = tab.drop(['month_s', 'day_s','month_e','day_e'], axis=1)
#quando la differenza tra l'anno di inizio e fine prestito è pari a zero, il prestito 'nasce' e 'muore' nel medesimo anno.
#questo comporta che per tale anno si rifletta l'intero valore del prestito
years_0 = pre_process.loc[(pre_process['years'] == 0)]
years_0['year_loan']= years_0['year_start']
years_0[['year_loan','loan_amount']]
loans_year_0 = years_0.groupby(years_0['year_loan'])['loan_amount'].sum().to_frame().reset_index()
loans_0=loans_year_0.rename(columns={'loan_amount':'loan_0'})
#quando la differenza è pari a 1 il prestito è a cavallo tra due anni, quindi tenendo conto di quanti giorni siano passati nei
#rispettivi anni, calcolo l'ammontare del prestito  
years_1 = pre_process.loc[(pre_process['years'] == 1)]
x=years_1['delta_start']
y=years_1['delta_end']
years_1['ds']=x/ np.timedelta64(1, 'D')
years_1['de']=y/np.timedelta64(1,'D')
years_1['first_year_loan']= years_1['year_start']
years_1['second_year_loan']=years_1['year_end']
years_1['loan_first_year']= (years_1['loan_amount']*years_1['ds'])/years_1['ds']+years_1['de']
years_1['loan_second_year']= (years_1['loan_amount']*years_1['de'])/years_1['ds']+years_1['de']
temp=years_1[['first_year_loan','loan_first_year','second_year_loan','loan_second_year']]
temp=temp.replace([np.inf, -np.inf], np.nan).fillna(0)
loans_year_1 = temp.groupby(temp["first_year_loan"])["loan_first_year"].sum().to_frame().reset_index()

years_1_1= temp[['second_year_loan','loan_second_year']]
years_1_1

loans_year_1_1 = years_1_1.groupby(years_1_1["second_year_loan"])["loan_second_year"].sum().to_frame().reset_index()
loans_year_1_1

merge = pd.merge(loans_year_1,loans_year_1_1, how='left',left_on='first_year_loan',right_on='second_year_loan').fillna(0)
merge['loan_amount']=merge['loan_first_year']+merge['loan_second_year']
merge.rename(columns={"first_year_loan":"year_loan",'loan_amount':'loan_1'},inplace=True)
loans_1=merge[['year_loan','loan_1']]

#la differenza tra gli anni di inizio e fine prestito equivalente a 2 e 3 non è presente 
#(pre_process.years[pre_process.years == x].count(), utilizzando tale codice si può notare come la somma di valori aventi come valori 'years' pari a 2 o 3 sia zero)

#calcolo il valore dell'ammontare dei prestiti relativo alla differenza tra gli anni di inizio e fine prestito pari a 4
years_4 = pre_process.loc[(pre_process['years'] == 4)]
x=years_4['delta_start']
y=years_4['delta_end']
years_4['ds']=x/ np.timedelta64(1, 'D')
years_4['de']=y/np.timedelta64(1,'D')
years_4['loan_first_year']= (years_4['loan_amount']*years_4['ds'])/years_4['ds']+years_4['de']+365+365+365
years_4['loan_second_year']=  (years_4['loan_amount']*365)/years_4['ds']+years_4['de']+365+365+365
years_4['loan_third_year']=  (years_4['loan_amount']*365)/years_4['ds']+years_4['de']+365+365+365
years_4['loan_fourth_year']= (years_4['loan_amount']*365)/years_4['ds']+years_4['de']+365+365+365
years_4['loan_fifth_year']=  (years_4['loan_amount']*years_4['de'])/years_4['ds']+years_4['de']+365+365+365
years_4['first_year_loan']= years_4['year_start']
years_4['second_year_loan']=years_4['first_year_loan']+1
years_4['third_year_loan']=years_4['first_year_loan']+2
years_4['fourth_year_loan']=years_4['first_year_loan']+3
years_4['fifth_year_loan']=years_4['year_end']
years_4[['first_year_loan','loan_first_year','second_year_loan','loan_second_year','third_year_loan','loan_third_year','fourth_year_loan','loan_fourth_year','fifth_year_loan','loan_fifth_year']]

loans_year_4 = years_4.groupby(years_4['first_year_loan'])['loan_first_year'].sum().to_frame().reset_index()
loans_year_4.rename(columns={'first_year_loan':'year_loan','loan_first_year':'loan_4'},inplace=True)
loans_year_4_1 = years_4.groupby(years_4['second_year_loan'])['loan_second_year'].sum().to_frame().reset_index()
loans_year_4_1.rename(columns={'second_year_loan':'year_loan','loan_second_year':'loan_4'},inplace=True)
loans_year_4_2 = years_4.groupby(years_4['third_year_loan'])['loan_third_year'].sum().to_frame().reset_index()
loans_year_4_2.rename(columns={'third_year_loan':'year_loan','loan_third_year':'loan_4'},inplace=True)
loans_year_4_3 = years_4.groupby(years_4['fourth_year_loan'])['loan_fourth_year'].sum().to_frame().reset_index()
loans_year_4_3.rename(columns={'fourth_year_loan':'year_loan','loan_fourth_year':'loan_4'},inplace=True)
loans_year_4_4 = years_4.groupby(years_4['fifth_year_loan'])['loan_fifth_year'].sum().to_frame().reset_index()
loans_year_4_4.rename(columns={'fifth_year_loan':'year_loan','loan_fifth_year':'loan_4'},inplace=True)
loans_4=pd.concat([loans_year_4,loans_year_4_1,loans_year_4_2,loans_year_4_3,loans_year_4_4], ignore_index=True)

#calcolo il valore dell'ammontare dei prestiti relativo alla differenza tra gli anni di inizio e fine prestito pari a 5

years_5 = pre_process.loc[(pre_process['years'] == 5)]
x=years_5['delta_start']
y=years_5['delta_end']
years_5['ds']=x/ np.timedelta64(1, 'D')
years_5['de']=y/np.timedelta64(1,'D')
years_5['loan_first_year']= (years_5['loan_amount']*years_5['ds'])/years_5['ds']+years_5['de']+365+365+365+365
years_5['loan_second_year']=  (years_5['loan_amount']*365)/years_5['ds']+years_5['de']+365+365+365+365
years_5['loan_third_year']=  (years_5['loan_amount']*365)/years_5['ds']+years_5['de']+365+365+365+365
years_5['loan_fourth_year']= (years_5['loan_amount']*365)/years_5['ds']+years_5['de']+365+365+365+365
years_5['loan_fifth_year']= (years_5['loan_amount']*365)/years_5['ds']+years_5['de']+365+365+365+365
years_5['loan_sixth_year']=  (years_5['loan_amount']*years_5['de'])/years_5['ds']+years_5['de']+365+365+365+365
years_5['first_year_loan']= years_5['year_start']
years_5['second_year_loan']=years_5['first_year_loan']+1
years_5['third_year_loan']=years_5['first_year_loan']+2
years_5['fourth_year_loan']=years_5['first_year_loan']+3
years_5['fifth_year_loan']=years_5['first_year_loan']+4
years_5['sixth_year_loan']=years_5['year_end']
years_5[['first_year_loan','loan_first_year','second_year_loan','loan_second_year','third_year_loan','loan_second_year','fourth_year_loan','loan_fourth_year','fifth_year_loan','loan_fifth_year','sixth_year_loan','loan_sixth_year']]

loans_year_5 = years_5.groupby(years_5['first_year_loan'])['loan_first_year'].sum().to_frame().reset_index()
loans_year_5.rename(columns={'first_year_loan':'year_loan','loan_first_year':'loan_5'},inplace=True)
loans_year_5_1 = years_5.groupby(years_5['second_year_loan'])['loan_second_year'].sum().to_frame().reset_index()
loans_year_5_1.rename(columns={'second_year_loan':'year_loan','loan_second_year':'loan_5'},inplace=True)
loans_year_5_2 = years_5.groupby(years_5['third_year_loan'])['loan_third_year'].sum().to_frame().reset_index()
loans_year_5_2.rename(columns={'third_year_loan':'year_loan','loan_third_year':'loan_5'},inplace=True)
loans_year_5_3 = years_5.groupby(years_5['fourth_year_loan'])['loan_fourth_year'].sum().to_frame().reset_index()
loans_year_5_3.rename(columns={'fourth_year_loan':'year_loan','loan_fourth_year':'loan_5'},inplace=True)
loans_year_5_4 = years_5.groupby(years_5['fifth_year_loan'])['loan_fifth_year'].sum().to_frame().reset_index()
loans_year_5_4.rename(columns={'fifth_year_loan':'year_loan','loan_fifth_year':'loan_5'},inplace=True)
loans_year_5_5 = years_5.groupby(years_5['sixth_year_loan'])['loan_sixth_year'].sum().to_frame().reset_index()
loans_year_5_5.rename(columns={'sixth_year_loan':'year_loan','loan_sixth_year':'loan_5'},inplace=True)
loans_5=pd.concat([loans_year_5,loans_year_5_1,loans_year_5_2,loans_year_5_3,loans_year_5_4,loans_year_5_5], ignore_index=True)

#dai dataframe risultanti da ogni calcolo pongo il medesimo indice, ovvero 'year_loan'
loans_0.set_index('year_loan',inplace=True)
loans_1.set_index('year_loan',inplace=True)
loans_4.set_index('year_loan',inplace=True)
loans_5.set_index('year_loan',inplace=True)

#concateno i dataframe
final = pd.concat([loans_0,loans_1,loans_4,loans_5],axis=1,sort=False).reset_index()
#sommo i risultati rispettando gli anni dei prestiti
final['loan_amount'] = final.loan_0.fillna(0) + final.loan_1.fillna(0)+final.loan_4.fillna(0)+final.loan_5.fillna(0)
final[['year_loan','loan_amount']]



,year_loan,loan_amount
0,2011,2.781085e+06
1,2012,1.429321e+08
2,2013,1.703100e+08
3,2014,2.189640e+08
4,2015,2.213673e+08
5,2016,2.166643e+08
6,2017,2.347679e+08
7,2018,8.530000e+04
